---
**EJERCICIO:** en data science y programación científica es común necesitar vectores con valores dentro de cierto intervalo, pero con sub-intervalos internos con diference espaciamiento ,por ejemplo:

[0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ]
       
El primer sub-intervalo incrementa de 0.1 en 0.1 , el segundo de 0.01 en 0.01 , y el tercero de 0.1 en 0.1 nuevamente.

Usando unicamente NumPy crea un vector de este tipo con el nombre xs .

**nota**: no es permitido usar ciclos, debe realizarse con operaciones "vectorizadas" de numpy

In [1]:
import numpy as np

print ( np.concatenate( [np.linspace(0, 0.5, 6), np.linspace(0.51, 0.59, 9), np.linspace(0.6, 1, 5)] ) ) 

[0.   0.1  0.2  0.3  0.4  0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58
 0.59 0.6  0.7  0.8  0.9  1.  ]


---
**EJERCICIO:** Para calcular la información de una distribución de probabilidad necesitamos calcular el producto entre la información de cada posible x y su probabilidad, cada uno de estos elementos nos indica cuanta incerteza aporta cada x a la entropía de la distribución. Crea una función que calcule esto, debe recibir un vector representando a la dist. de probabilidad y devoler otro vector con el termino de entropía para cada x. Por ejemplo:

calcular_entropia([0.25,0.75])

Debe resultar en :

[0.5    ,    0.31127812]

**Nota**: la entropía es la suma sobre estos valores, en este caso no calculamos la entropía,unicamente los termimos de su sumatoria. La entropía es:

$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

In [2]:
def calcular_entropia(dist_probabilidad):
    return -1 * np.array(dist_probabilidad) * np.log2(dist_probabilidad)
    
print (calcular_entropia([0.25,0.75]))

[0.5        0.31127812]


---
#### Aritmetica acumulativa

En muchos casos es útil usar sumas o productos acumulativos , por ejemplo:

**Ejemplo aplicado** En un datawarehouse se tiene un modelo "lifetime value" en el que se almacena de manera acumulativa cuantas ganancias se han percibido a lo largo de la vida de los usuarios, un ETL en Python debe calcularlo en función de un vector que contiene de manera ordenada en el tiempo los ingresos percibidos mensualmente para cierto usuario. Podemos usar la funcion **np.cumsum** para esto, en este ejercicio necesitamos determinar el "lifetime value" de cierto usuario en su 6to mes:

In [3]:
def lifetime_value(ganancias_mensuales):
    return np.cumsum(ganancias_mensuales)

lifetime_value_usuario = lifetime_value( np.array([25,150,125,75,100,45,22,30]) )
print("Lifetime value:", lifetime_value_usuario)
print("El valor del usuario a su sexto mes es :", lifetime_value_usuario[5])

Lifetime value: [ 25 175 300 375 475 520 542 572]
El valor del usuario a su sexto mes es : 520


---
**EJERCICIO:** Usando argmax crear una función mode(x) para calcular la moda de un vector x , esta función debe devolver una tupla de 2 elementos de la forma:

(valor,conteo)

**tip** investigar y auxiliarse de otras funciones de numpy y np.argmax, no olvidar trabajar todo vectorizado, sin ciclos.

In [71]:
def mode(values):
    uniques, counts = np.unique(values, return_counts=True)
    max_freq_index = np.argmax(counts)
    max_freq_value = uniques[max_freq_index]
    return (max_freq_value, counts[max_freq_index])

mode(np.array([9,8,6,5,4,3,9,12,16,0,9,0,14,11,10,9,1,2,3,4,5,9]))

(9, 5)

---

**EJERCICIO:** crear una función para calcular la entropía  de una distribución de probabilidad representada como un vector.
$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

**Nota**: a diferencia del ejercicio anterior donde solo calculamos los términos de la entropía, en este caso  si calculamos el valor completo de la entropía, que como mencionamos es una medida de incerteza en una distribución de probabilidad.

**recordatorio**: no usar ciclos solo operaciones vectorizadas.

In [65]:
# Reutilizamos la función que ya habíamos creado
print ( -1 * np.sum(calcular_entropia([0.25, 0.75])) )

-0.8112781244591328


---
**EJERCICIO:** En ML usamos la entropía cruzada como una forma de medir que tan bueno es un modelo de variables discretas a través de comparar la distribución de probabilidad que el modelo produce o predice, vs la distribución de probabilidad real dada por los datos de entrenamiento.

Podemos ver el siguiente ejemplo que define la forma en que se calcula la entropía cruzada y nos muestra un caso específico:

<img src="https://image.slidesharecdn.com/publishintroductiontodeeplearninginpythonandmatlab1-160502102437/95/introduction-to-deep-learning-in-python-and-matlab-54-638.jpg?cb=1462185644">

En este caso interpretamos así: El problema consiste en un modelo o algoritmo de ML que debe producir un vector de 3 elementos indicando la probabilidad de que ciertos datos X pertenezcan a una de 3 categorías.

* El modelo de ML produce un vector que indica que estima un 70% de probabilidad de que se trate de la categoría 0, 20% de probabilidad de que se trate de la categoría 1 y 10% de que se trate de la categoría 2.
* Los datos reales nos dicen que se trataba de un caso donde con total certeza se sabe que se trata de la categoría 0
* La entropía cruzada(a calcular en el ejercicio) nos indica que tan buena es la estimación del modelo, una EC de 0 es un modelo perfecto(en este caso un modelo que predice 100% de prob para la clase 0)

**Nota** 
* Aun que para calcular la entropía usamos logaritmos en base 2, en ML para calcular la entropía cruzada se usa logaritmo natural ya que con este se cumple el proposito **estimar que tanto se alejan las predicciones del modelo de ML de los datos reales** y es comunmente mas rápido de calcular en la computadora.
* Ya que estamos trabajando con vectores que representan distribuciones de probabilidad , podemos toparnos con lo que se conoce como : **sparse vectors**(vectores donde la mayoría de elementos son 0), esto puede producir problemas ya que le logaritmo de 0 no esta definido, tu solución debe tomar en cuenta esto y evitar que devuelva "nan"

In [112]:
def cross_entropy(y, y_hat):
    has_nan = np.sum(np.isnan(np.log(y_hat)))
    return -1 * np.sum(y * np.log(y_hat)) if (has_nan==0) \
            else "El segundo array tiene valores no aplicables para logaritmo natural"

y  = np.array([1.0,0,0])
y_hat = np.array([0.7,0.2,-0.1]) # <- resultado de algun modelo predictivo

cross_entropy(y,y_hat)

<ipython-input-112-3c6e93bde786>:2: RuntimeWarning: invalid value encountered in log
  has_nan = np.sum(np.isnan(np.log(y_hat)))


'El segundo array tiene valores no aplicables para logaritmo natural'

---
### Magnitud de un Vector(norma o módulo)
Ya hemos visto que en DS usamos vectores para representar información ,una propiedad importante de los vectores es la **magnitud** del vector. 

Matemáticamente la magnitud de un vector(también conocida como norma vectorial)  nos indica  el tamaño de este , y nos sirve para tener una noción de la distancia desde un punto de referencia(origen) hasta el punto representado por el vector.

**EJERCICIO:** crear una función que reciba como parámetro un vector x y calcule su magnitud o norma(euclidiana o L2) ,luego usarla para evaluar 2 vectores que representan los errores generados por 2 modelos de machine learning y concluir cual de los 2 modelos es mejor.

In [115]:
def magnitud(x):
    np.sum(np.absolute(x)) # valor absoluto
    return np.sum(np.power(x,2)) # raiz cuadrada

errores_modelo1 = np.array([1,2,1,2]) #
errores_modelo2 = np.array([0,1,1,2]) #

vector1_size = magnitud(errores_modelo1)
vector2_size = magnitud(errores_modelo2)

print( "El primer vector es mejor" if (vector1_size < vector2_size) else "El segundo vector es mejor" )

El segundo vector es mejor


----
**EJERCICIO:** Usando la función del ejercicio anterior, crea otra función normalizar(x) que reciba de parámetro un vector x aplique normalización sobre este, el resultado debe ser un nuevo vector del tamaño de x cuya magnitud es igual a 1.0

In [117]:
def normalizar(x):
    return x/np.linalg.norm(x)   

print(magnitud(normalizar(errores_modelo1)))
print(normalizar(errores_modelo2))

1.0
[0.         0.40824829 0.40824829 0.81649658]


---
**EJERCICIO:**
Se tiene una red neuronal sencilla(y simplificada):

Donde:
* INPUT LAYER: un vector X de tamaño = 2 que representa los datos de entrada
* HIDDEN_LAYER :capa oculta con 2 neuronas definidas por los vectores:
    * HL1 = [0.25,0.37]
    * HL2 = [-8,14]
* OUTPUT_LAYER = capa de salida definida por el vector [4,9]

Crear una funcion neural_network(X) para calcular:
* Calcule la salida de cada neurona en la capa intermedia aplicada a la capa de entrada.
* Use el resultado del paso anterior como entrada para la neurona en la capa de salida

Asumiendo que cada neurona identifica la similitud entre su entrada y la caracteística que representa concluir:
* Para cada vector de entrada Xi , cual neurona intermedia busca la característica que mas se parece a X.
* Cual vector de entrada Xi produce una activación alta(salida alta) en la capa de salida.

In [121]:
# Aplicar la red neuronal sobre los siguientes datos X
def salida_intermedia(v, hidden_layer):
    return np.dot(v, hidden_layer)
    
hl1 = [0.25,0.37]
hl2 = [-8,14]
out_layer = [-8,14]

X1 = np.array([0.50,0.72])
X2 = np.array([-4,7])
X3 = np.zeros_like(X2)
x4 = np.ones_like(X1)
X5 = np.random.randn(X1.shape[0])

salida1 = salida_intermedia(X1, hl1)
salida2 = salida_intermedia(X1, hl2)
print("Salida de cada neurona en la capa intermedia aplicada a la capa de entrada:")
print(salida1)
print(salida2)

print("\nCapa de salida:")
print(salida_intermedia([salida1, salida2], out_layer))

Salida de cada neurona en la capa intermedia aplicada a la capa de entrada:
0.39139999999999997
6.08

Capa de salida:
81.9888
